## Initializing everything firstly

In [1]:
from pyspark.sql import SparkSession

In [2]:
# Initialize a Spark session
spark = SparkSession.builder.appName("DataTransformation").getOrCreate()

23/09/06 12:50:52 WARN Utils: Your hostname, kushal-Latitude-E5440 resolves to a loopback address: 127.0.1.1; using 172.16.5.134 instead (on interface wlp2s0)
23/09/06 12:50:52 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/09/06 12:50:53 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
#loading dataframes with clean data
listing_clean = spark.read.csv('cleaned_data/clean_listing_csv/', header=True, inferSchema=True)
reviews_clean = spark.read.csv('cleaned_data/clean_reviews_csv/', header=True, inferSchema=True)
calendar_clean = spark.read.csv('cleaned_data/clean0_calendar_csv/', header=True, inferSchema=True)

## UDF Problem: Create a UDF to perform sentiment analysis on the review comments and categorize them as positive, neutral, or negative. Apply this UDF to reviews_df to analyze the sentiment of reviews.

In [5]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
from textblob import TextBlob

In [6]:
# Define a function to perform sentiment analysis using TextBlob
def analyze_sentiment(comment):
    analysis = TextBlob(comment)
    # Classify sentiment as positive, neutral, or negative based on polarity
    if analysis.sentiment.polarity > 0:
        return "positive"
    elif analysis.sentiment.polarity == 0:
        return "neutral"
    else:
        return "negative"

# Register the UDF
sentiment_analysis_udf = udf(analyze_sentiment, StringType())

In [7]:
from pyspark.sql.functions import col

reviews_sentiment = reviews_clean.withColumn("sentiment", sentiment_analysis_udf(col("comments")))

In [8]:
reviews_sentiment.show(50, truncate=False)

+----------+--------+----------+-----------+-------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

## Section 2: Using groupBy, calculate the total number of reviews left for Boston listings each month. Rename the resulting columns to "Month" and "Total Reviews."

In [9]:
from pyspark.sql.functions import to_date
from pyspark.sql.types import DateType

# Assuming 'date_string' is in the format 'yyyy-MM-dd'
date_format = 'yyyy-MM-dd'

# Use the to_date function to convert the string column to a date column
reviews_clean_1 = reviews_clean.withColumn("date", to_date(reviews_clean["date"], date_format).cast(DateType()))

In [10]:
from pyspark.sql.functions import month, date_format

reviews_U2 = reviews_clean_1.withColumn("month", month(reviews_clean_1["date"]))
#reviews_U2 = reviews_U2.withColumn("month_name", date_format(reviews_U2["month"], "MMM"))

# Group by Month, and count the number of reviews
monthly_reviews = reviews_U2.groupBy("Month") \
    .count() \
    .withColumnRenamed("count", "Total Reviews") \
    .orderBy("Month")

# Show the resulting DataFrame
monthly_reviews.show()

+-----+-------------+
|Month|Total Reviews|
+-----+-------------+
|    1|         2119|
|    2|         1964|
|    3|         3853|
|    4|         6116|
|    5|         8115|
|    6|         8369|
|    7|         9333|
|    8|        10345|
|    9|         5985|
|   10|         6095|
|   11|         3729|
|   12|         2252|
+-----+-------------+



## Window function : Identify significant popularity changes in Airbnb listings in Boston based on month-to-month booking. 

In [13]:
from pyspark.sql.window import Window
from pyspark.sql.functions import to_date, month, count

calendar_window = calendar_clean.withColumn("date", to_date(calendar_clean["date"]))

# Calculate the total number of bookings per listing
bookings_count = calendar_window.filter(calendar_clean["available"] == False) \
    .groupBy("listing_id") \
    .agg(count("listing_id").alias("total_bookings"))

# Calculate the number of bookings per listing for each month
bookings_count = calendar_window.filter(calendar_window["available"] == False) \
    .groupBy("listing_id", month("date").alias("month")) \
    .agg(count("listing_id").alias("bookings"))

In [14]:
bookings_count.show()

+----------+-----+--------+
|listing_id|month|bookings|
+----------+-----+--------+
|   1436513|   11|      22|
|   6400432|   12|      31|
|  12426823|    3|      31|
|   6862758|   11|      30|
|     27611|    5|      31|
|   1754939|    3|      31|
|   3905508|    4|       5|
|   5182313|   10|       2|
|  13698137|    1|      31|
|  13565466|    4|      30|
|  13000520|    8|      31|
|   4409653|    6|      30|
|  12986892|    3|      31|
|   7434871|    7|      31|
|  14222649|    4|      30|
|  13248823|    4|      30|
|  14035943|   12|      31|
|    766700|   10|      28|
|   3434614|    2|      28|
|  14624254|   12|      30|
+----------+-----+--------+
only showing top 20 rows



In [15]:
from pyspark.sql.functions import lag, when, isnull
window_spec = Window.partitionBy("listing_id").orderBy("month")
monthly_bookings = bookings_count.withColumn("previous_month_bookings", lag("bookings").over(window_spec))

In [16]:
monthly_bookings1 = monthly_bookings.withColumn("popularity_change", when(isnull(monthly_bookings["previous_month_bookings"]), 0)
                                                .otherwise(monthly_bookings["bookings"] - monthly_bookings["previous_month_bookings"]))

In [17]:
from pyspark.sql.functions import abs
significant_popularity_changes = monthly_bookings1.filter(abs("popularity_change") >= 0.5)
significant_popularity_changes.show()

+----------+-----+--------+-----------------------+-----------------+
|listing_id|month|bookings|previous_month_bookings|popularity_change|
+----------+-----+--------+-----------------------+-----------------+
|      3353|   10|      31|                     25|                6|
|      3353|   11|      30|                     31|               -1|
|      5506|   10|       7|                     14|               -7|
|      6695|   10|      18|                     23|               -5|
|      6976|   10|       3|                     11|               -8|
|      6976|   11|      11|                      3|                8|
|      6976|   12|      21|                     11|               10|
|      8792|   10|      31|                     25|                6|
|      8792|   11|      30|                     31|               -1|
|      8792|   12|      31|                     30|                1|
|      9824|   10|      11|                     17|               -6|
|      9824|   11|  

## Pivot:You are given two DataFrames: listings_df and calendar_df, containing information about Airbnb listings and their availability in Boston over time. Your task is to analyze the demand for different property types in Boston by calculating the count of bookings for each property type. 

In [18]:
calendar_cleanp = calendar_clean.filter(calendar_clean["available"]==False)

In [19]:
calendar_cleanp.show()

+----------+----------+---------+-----+
|listing_id|      date|available|price|
+----------+----------+---------+-----+
|  12147973|2017-09-05|    false| null|
|  12147973|2017-09-04|    false| null|
|  12147973|2017-09-03|    false| null|
|  12147973|2017-09-02|    false| null|
|  12147973|2017-09-01|    false| null|
|  12147973|2017-08-31|    false| null|
|  12147973|2017-08-30|    false| null|
|  12147973|2017-08-29|    false| null|
|  12147973|2017-08-28|    false| null|
|  12147973|2017-08-27|    false| null|
|  12147973|2017-08-26|    false| null|
|  12147973|2017-08-25|    false| null|
|  12147973|2017-08-24|    false| null|
|  12147973|2017-08-23|    false| null|
|  12147973|2017-08-22|    false| null|
|  12147973|2017-08-21|    false| null|
|  12147973|2017-08-20|    false| null|
|  12147973|2017-08-19|    false| null|
|  12147973|2017-08-18|    false| null|
|  12147973|2017-08-17|    false| null|
+----------+----------+---------+-----+
only showing top 20 rows



In [20]:
property_demand = listing_clean.join(calendar_cleanp, listing_clean["id"] == calendar_cleanp["listing_id"]) \
    .groupBy("property_type") \
    .agg(count("listing_id").alias("booking_count"))

In [21]:
property_demand = property_demand.withColumn("Demand_Category",
                                             when(property_demand["booking_count"] > 1000, "High Demand")
                                             .otherwise("Low Demand"))

In [22]:
property_demand.show()

+---------------+-------------+---------------+
|  property_type|booking_count|Demand_Category|
+---------------+-------------+---------------+
|      Apartment|       314353|    High Demand|
|      Townhouse|         7704|    High Demand|
|Bed & Breakfast|         3516|    High Demand|
|           Loft|         5934|    High Demand|
|     Guesthouse|          206|     Low Demand|
|          Villa|          763|     Low Demand|
|   Entire Floor|          609|     Low Demand|
|    Condominium|        33059|    High Demand|
|          House|        59016|    High Demand|
|           Boat|          703|     Low Demand|
|           null|          582|     Low Demand|
|      Camper/RV|           26|     Low Demand|
|          Other|          519|     Low Demand|
|           Dorm|          365|     Low Demand|
+---------------+-------------+---------------+



In [23]:
pivot_table = property_demand.groupBy("property_type") \
    .pivot("Demand_Category", ["High Demand", "Low Demand"]) \
    .agg(count("property_type")) \
    .withColumnRenamed("High Demand", "IS_High_Demand") \
    .withColumnRenamed("Low Demand", "IS_Low_Demand")

In [24]:
pivot_table = pivot_table.withColumn("IS_High_Demand", when(col("IS_High_Demand") > 0, True).otherwise(False))
pivot_table = pivot_table.withColumn("IS_Low_Demand", when(col("IS_Low_Demand") > 0, True).otherwise(False))

In [25]:
pivot_table.show()

+---------------+--------------+-------------+
|  property_type|IS_High_Demand|IS_Low_Demand|
+---------------+--------------+-------------+
|      Apartment|          true|        false|
|      Townhouse|          true|        false|
|Bed & Breakfast|          true|        false|
|           Loft|          true|        false|
|     Guesthouse|         false|         true|
|          Villa|         false|         true|
|   Entire Floor|         false|         true|
|    Condominium|          true|        false|
|          House|          true|        false|
|           Boat|         false|         true|
|      Camper/RV|         false|         true|
|          Other|         false|         true|
|           Dorm|         false|         true|
|           null|         false|        false|
+---------------+--------------+-------------+



## PIVOT: How many property are there in each category that have more or less than 50 reviews

In [26]:
reviews_clean.printSchema()

root
 |-- listing_id: integer (nullable = true)
 |-- id: integer (nullable = true)
 |-- date: date (nullable = true)
 |-- reviewer_id: integer (nullable = true)
 |-- reviewer_name: string (nullable = true)
 |-- comments: string (nullable = true)



In [27]:
reviews_count = reviews_clean.groupBy("listing_id").agg(count("*").alias("total_reviews_count"))

In [28]:
reviews_count.show()

+----------+-------------------+
|listing_id|total_reviews_count|
+----------+-------------------+
|   7825327|                  9|
|   2754975|                 30|
|   8165898|                  3|
|   4149974|                 70|
|   6006328|                 10|
|  14322091|                  1|
|   5991427|                 23|
|  12796034|                  1|
|   8536113|                  9|
|   6955911|                  5|
|  13282275|                  1|
|   5783197|                 35|
|  13236936|                  1|
|   6489209|                 14|
|   9169272|                  1|
|    979759|                  2|
|     31796|                291|
|   1178371|                 85|
|   1067184|                 53|
|   6453686|                 58|
+----------+-------------------+
only showing top 20 rows



In [29]:
final_df = listing_clean.join(reviews_count, listing_clean["id"] == reviews_count["listing_id"])

In [30]:
reviews_df = final_df.withColumn('review_category', when(col('number_of_reviews') < 50, 'less_than_50').otherwise('greater_than_50'))

# Group by property type and review category, and calculate the count of reviews
review_pivot = reviews_df.groupBy('property_type') \
    .pivot('review_category', values=['less_than_50', 'greater_than_50']) \
    .agg(count('id')) \
    .na.fill(0)

# Show the result
review_pivot.show()

+---------------+------------+---------------+
|  property_type|less_than_50|greater_than_50|
+---------------+------------+---------------+
|      Apartment|        1254|            232|
|      Townhouse|          29|              5|
|Bed & Breakfast|          19|              9|
|           null|           2|              0|
|           Loft|          25|              2|
|     Guesthouse|           1|              0|
|          Villa|           2|              0|
|          Other|           8|              0|
|           Dorm|           1|              0|
|    Condominium|         130|             14|
|          House|         283|             97|
|           Boat|          11|              0|
+---------------+------------+---------------+

